### **MOUNTING DRIVE TO ACCESS AND SAVE DATA**

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **IMPORTING RELEVANT MODULES**

In [30]:
import numpy as np
import pandas as pd
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import load_model
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### **TOKENIZING AND PREPROCESSING TWEETS**

In [31]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if type(text) != float:
        tokens = word_tokenize(text.lower())
        filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
        return ' '.join(filtered_tokens)
    else:
        return str(text)

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/extern.csv')

df['preprocessed_tweet'] = df['tweet'].apply(preprocess_text)
X = df['preprocessed_tweet'].values
y = df['value'].values

### **DIVIDING INTO TRAIN AND VALIDATION SET**

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vocab_size = 50000  # Maximum number of words in the vocabulary
embedding_dim = 100
max_length = 50  # Maximum tweet length


tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post', truncating='post')

### **CREATION AND TESTING OF THE MODEL**

In [ ]:
model1 = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64),
    Dense(1, activation='tanh')  # Output activation function can be changed based on your needs
])



model1.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model1.fit(X_train_padded, y_train, epochs=15,batch_size = 10,validation_data=(X_test_padded, y_test))

# Evaluate model1
loss, mae = model1.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss}, accuracy : {mae}")



Epoch 1/15
800/800 [==============================] - 76s 93ms/step - loss: 0.5981 - accuracy: 0.3670 - val_loss: 0.4893 - val_accuracy: 0.4990
Epoch 2/15
800/800 [==============================] - 74s 93ms/step - loss: 0.3348 - accuracy: 0.5971 - val_loss: 0.3737 - val_accuracy: 0.5455
Epoch 3/15
800/800 [==============================] - 74s 93ms/step - loss: 0.1539 - accuracy: 0.6842 - val_loss: 0.3667 - val_accuracy: 0.5715
Epoch 4/15
800/800 [==============================] - 73s 92ms/step - loss: 0.0880 - accuracy: 0.7190 - val_loss: 0.3692 - val_accuracy: 0.5710
Epoch 5/15
800/800 [==============================] - 75s 94ms/step - loss: 0.0673 - accuracy: 0.7287 - val_loss: 0.4040 - val_accuracy: 0.6095
Epoch 6/15
800/800 [==============================] - 74s 93ms/step - loss: 0.0794 - accuracy: 0.7308 - val_loss: 0.4952 - val_accuracy: 0.6070
Epoch 7/15
800/800 [==============================] - 72s 91ms/step - loss: 0.1094 - accuracy: 0.6894 - val_loss: 0.3997 - val_accuracy:

In [ ]:
model2 = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(1, activation='tanh')  # Output activation function can be changed based on your needs
])

model2.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model2.fit(X_train_padded, y_train, epochs=5,validation_data=(X_test_padded, y_test))

# Evaluate model2
loss, mae = model2.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss}, accuracy: {mae}")

In [34]:
test_model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='tanh')
])

test_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
test_model.fit(X_train_padded, y_train, epochs=100,batch_size=40,validation_data=(X_test_padded, y_test))

# Evaluate test_model
loss, mae = test_model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss}, accuracy: {mae}")

Epoch 1/100
200/200 [==============================] - 119s 345ms/step - loss: 0.6223 - accuracy: 0.3503 - val_loss: 0.6239 - val_accuracy: 0.3375
Epoch 2/100
200/200 [==============================] - 72s 361ms/step - loss: 0.6101 - accuracy: 0.3801 - val_loss: 0.5376 - val_accuracy: 0.5260
Epoch 3/100
200/200 [==============================] - 67s 337ms/step - loss: 0.3580 - accuracy: 0.5656 - val_loss: 0.3608 - val_accuracy: 0.5425
Epoch 4/100
200/200 [==============================] - 68s 337ms/step - loss: 0.1542 - accuracy: 0.6805 - val_loss: 0.3611 - val_accuracy: 0.5980
Epoch 5/100
200/200 [==============================] - 71s 356ms/step - loss: 0.0934 - accuracy: 0.7149 - val_loss: 0.3660 - val_accuracy: 0.5885
Epoch 6/100
200/200 [==============================] - 69s 347ms/step - loss: 0.1419 - accuracy: 0.7055 - val_loss: 0.4496 - val_accuracy: 0.5625
Epoch 7/100
200/200 [==============================] - 67s 336ms/step - loss: 0.1027 - accuracy: 0.6992 - val_loss: 0.3947 

### **SAVING THE MODEL**

In [ ]:

model1.save('/content/drive/MyDrive/Colab Notebooks/model1_15epoch.h5')
print("Model1 saved successfully!")
model2.save('/content/drive/MyDrive/Colab Notebooks/model2_5epoch.h5')
print("Model2 saved successfully!")
tokenizer_filename = "/content/drive/MyDrive/Colab Notebooks/tokenizer_15epoch.pkl"
with open(tokenizer_filename, 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)
print("tokenizer saved successfully!")

Model1 saved successfully!
tokenizer saved successfully!


In [35]:
test_model.save('/content/drive/MyDrive/Colab Notebooks/test.h5')
print("Model1 saved successfully!")
tokenizer_filename = "/content/drive/MyDrive/Colab Notebooks/tokenizertest.pkl"
with open(tokenizer_filename, 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)
print("tokenizer saved successfully!")

Model1 saved successfully!
tokenizer saved successfully!


### **MODEL LOADING AND TESTING USER INPUT**

In [ ]:
loaded_model1 = load_model('/content/drive/MyDrive/Colab Notebooks/waste/model1.h5')
loaded_model2 = load_model('/content/drive/MyDrive/Colab Notebooks/waste/model2.h5')

def predict_sentiment(tweet,model):
    preprocessed_tweet = preprocess_text(tweet)
    tweet_sequence = tokenizer.texts_to_sequences([preprocessed_tweet])
    padded_sequence = pad_sequences(tweet_sequence, maxlen=max_length, padding='post', truncating='post')
    sentiment_prediction = model.predict(padded_sequence)[0][0]
    return sentiment_prediction

for i in range(0,3):
    user_tweet = input("Enter a tweet: ")
    sentiment_prediction = predict_sentiment(user_tweet,loaded_model1)
    print("Sentiment prediction for model 1 :", sentiment_prediction)
    sentiment_prediction = predict_sentiment(user_tweet,loaded_model2)
    print("Sentiment prediction for model 2 :", sentiment_prediction)


Enter a tweet: People wanting more govt jobs is also a loss to exchequer coz it's not a business that makes money. This is more about tapping a niche in India and paving way for more advanced stuff.
1/1 [==============================] - 0s 422ms/step
Sentiment prediction for model 1 : 0.22203273
1/1 [==============================] - 1s 796ms/step
Sentiment prediction for model 2 : -0.00281773
Enter a tweet: For the BJP all transactions are a PR exercise that pulls  the wool over the eyes of the public. Even that is not necessary because the public is blind anyway. Few people are able to recognise the difference between transfer of core technology and exploitation of cheap labour.
1/1 [==============================] - 0s 34ms/step
Sentiment prediction for model 1 : 0.90512955
1/1 [==============================] - 0s 45ms/step
Sentiment prediction for model 2 : -0.9342344
Enter a tweet: How many semiconductor factories we have in india? How many are willing to build them here without